In [2]:
!wget https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/roman_numerals.py


--2022-02-03 00:28:32--  https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/roman_numerals.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2161 (2.1K) [text/plain]
Saving to: ‘roman_numerals.py’

roman_numerals.py   100%[===================>]   2.11K  --.-KB/s    in 0s      

2022-02-03 00:28:32 (27.1 MB/s) - ‘roman_numerals.py’ saved [2161/2161]



In [3]:
!wget https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/encoder.py


--2022-02-03 00:28:50--  https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/encoder.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1812 (1.8K) [text/plain]
Saving to: ‘encoder.py’

encoder.py          100%[===================>]   1.77K  --.-KB/s    in 0s      

2022-02-03 00:28:50 (37.3 MB/s) - ‘encoder.py’ saved [1812/1812]



In [4]:
import random
from tensorflow.python.keras import layers
from tensorflow.python.keras.models import Sequential
import roman_numerals as cnv
import numpy as np
import random

In [5]:
from encoder import CharacterTable 


In [6]:
# Parameters for the model and dataset.
DATASET_SIZE = 200
INPUT = 14
OUTPUT = 4
# object to encode roman numbers to one-hot 
romans = 'MDCLXVI '
rtable = CharacterTable(romans)

# object to encode arabic numbers to one-hot
chars = '0123456789 '
dtable = CharacterTable(chars)


In [7]:
seq_samples = []
seq_labels = []
used = []
repetitions = 0
generated = 0
print('Generating data...')
while len(seq_samples) < DATASET_SIZE:
    number = random.randint(1,200)
    # skip if already in the dataset
    if number in used: continue
    used.append(number)
    # roman input
    roman = cnv.convert(number) 
    roman += ' ' * (INPUT - len(roman))
    
    # arabic output
    arabic = str(number)
    arabic += ' ' * (OUTPUT - len(arabic)) 
    
    seq_samples.append(roman)
    seq_labels.append(arabic)
print('Total roman numbers:', len(seq_samples))
#print("samples",seq_samples)
#print("labels",seq_labels)

Generating data...
Total roman numbers: 200


In [8]:
# one-hot encoding of all romans and arabic numbers
print('Vectorization...')
samples = np.zeros((DATASET_SIZE, INPUT, len(romans)), dtype=np.bool)
labels = np.zeros((DATASET_SIZE, OUTPUT, len(chars)), dtype=np.bool)
for i, sentence in enumerate(seq_samples):
    samples[i] = rtable.encode(sentence, INPUT)
for i, sentence in enumerate(seq_labels):
    labels[i] = dtable.encode(sentence, OUTPUT)
print("samples",samples.shape)    
print("labels",labels.shape)

Vectorization...
samples (200, 14, 8)
labels (200, 4, 11)


In [9]:
import sklearn.model_selection
(trainSamples, testSamples, trainLabels, testLabels) = sklearn.model_selection.train_test_split(samples,labels,random_state=42)
print("Training samples",len(trainSamples))
print("Test samples",len(testSamples))

Training samples 150
Test samples 50


In [10]:
# Build the network
print('Build model...')
model = Sequential()
model.add(layers.LSTM(128, input_shape=(INPUT, len(romans))))
model.add(layers.RepeatVector(OUTPUT))
model.add(layers.LSTM(128, return_sequences=True))
model.add(layers.Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               70144     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense (Dense)                (None, 4, 11)             1419      
Total params: 203,147
Trainable params: 203,147
Non-trainable params: 0
_________________________________________________________________


In [11]:
def avg(list):
    return sum(list)/len(list)

EPOCHS = 50
# Train the model each generation and show predictions against the validation dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration*EPOCHS)
    H = model.fit(trainSamples, trainLabels, batch_size=128, epochs=EPOCHS, 
                  validation_data=(testSamples, testLabels), verbose=0)
    print("Validation accuracy: {}".format(avg(H.history["val_accuracy"])))
    # Select 10 samples from the validation set at random to visualize errors.
    accuracy = 0
    for i in range(len(testSamples)):
        
        rowx, rowy = testSamples[np.array([i])], testLabels[np.array([i])]
        preds = model.predict_classes(rowx, verbose=0)
        roman = rtable.decode(rowx[0])
        correct = dtable.decode(rowy[0])
        prediction = dtable.decode(preds[0], calc_argmax=False)
        correctTxt = ""
        if correct == prediction:
            correctTxt+='OK!'
            accuracy+=1
        v = random.random()
        if v>0.9:
            print('expression:', roman, end=' ')
            print('correct:', correct, end=' ')
            print('predicted:', prediction, end=' ')
            print(correctTxt)
        
    print('accuracy=',accuracy,"/",len(testSamples),"=",accuracy/len(testSamples))


--------------------------------------------------
Iteration 50
Validation accuracy: 0.47040000081062316


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:454: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


expression: CLXXXIX        correct: 189  predicted: 166  
expression: CLII           correct: 152  predicted: 16   
expression: XXXI           correct: 31   predicted: 16   
expression: XLVI           correct: 46   predicted: 16   
expression: CLXIV          correct: 164  predicted: 166  
accuracy= 0 / 50 = 0.0

--------------------------------------------------
Iteration 100
Validation accuracy: 0.5537000060081482
expression: CLXX           correct: 170  predicted: 190  
expression: CXLIV          correct: 144  predicted: 196  
expression: XLVI           correct: 46   predicted: 62   
expression: XLVII          correct: 47   predicted: 33   
expression: CVIII          correct: 108  predicted: 13   
expression: CXLVII         correct: 147  predicted: 177  
accuracy= 0 / 50 = 0.0

--------------------------------------------------
Iteration 150
Validation accuracy: 0.607700001001358
expression: XCIV           correct: 94   predicted: 19   
expression: LXVII          correct: 67   predic